# Lab 3: Search Terms with Pandas
### Created By: Christian Basso
___

This program revisits a previous term manipulation program. Although this lab will feature the same base data set, have similar function and outcomes, it will take advantage of the pandas library for data analysis instead of the built in python data strutures.
In addition, all function calls will be replaced with lambda functions to further explore data optimization.

Similar to the previous lab, this program will construct an spell check autocomplete function using Direct Supply's DSSI
eProcurement system. The DDSI eProcurement system allows for large data sets of common search terms within Direct Supply's website. So, this program is
highly tuned around their customers and products. This specific dataset contains searches entered by food-service users over a 60 day period from 2019.

In [23]:
import pandas as pd
import sys
from spellchecker import SpellChecker

ModuleNotFoundError: No module named 'spellchecker'

Similar to the previous program, the csv full of searches is read into a python list (although this program uses pandas, the initial import will be done with a normal list, as this was not a bottle neck in efficiency within the previous lab). This csv has a second column that includes a two letter unit code with a comma, this information is not needed for this program, so it its stripped before any information is altered.

In [2]:
searches = []
with open ('data/searchTerms.csv', 'r', encoding = 'utf8') as f:
    for line in f:
        searches.append(line.lower())
f.closed
searches.pop(0)
removed_code = []
for i in searches:
    removed_code.append(i.strip()[:-3])


Again, simialar to the last lab, this data set contains web spaces (which show up as %20) and normal spaces. The following code replaces web spaces with regular spaces. The replace function is used instead of the remove function to insure that purposful spaces are kept since each search will be split into single word tokens. This step is done before the data is inserted into the data frame to get each search into lists, which allows for easier manipulation later on (sometimes single word searches do not get put into a list when they are split in the dataframe).

Once the following code has run, split_searches will be a python list of single word tokens.

In [3]:
split_searches = []
for i in removed_code:
    split_searches.append(i.replace("%20", " ").split(" "))


Now that the data is in idividual tokens, a dataframe can be created. The dataframe will contain the list of tokens in the search (size depends on how many words were in the search). Although the data has been split, it still contains impropper spelling, characters, and punctuation. This will be taken care of later.

In [4]:
df = pd.DataFrame( {"Searches" : split_searches})
df.head(15)

,Searches
0,[36969]
1,"[cmed, 500100]"
2,"[kend, 5750]"
3,"[cmed, 980228]"
4,[dync1815h]
5,[dynd70642]
6,"[dees, kc-21400]"
7,"[link, pc1000]"
8,[7081714]
9,"[kend, 8507sa]"


As previously mentioned, each search token's punctuation, spelling, and charaters needs to be cleaned. In the last lab the python list full of tokens was sent through a number of functions to acomplish this. However, it was quite inefficent. In an atempt to optimimize the process, lambda functions will be applied the row of searches in the dataframe. The cleaning functions will be outlined first, then each lambda call will be made in individual cells. The cleaning functions are done before the spellcheck to ensure the spellcheck api does not get confused with numbers, punctuation, Etc. Each lambda function cell will be benchmarked.

The lambda function is completed by retriving the one column in the dataframe and applying the map function to each cleaning method.

In [5]:
column = df["Searches"]

The first cleaning function will be characters. This will take in one search and remove any unwanted characters (I.E. !@#$%^&*) and punctuation. This function also omits empty searches if removing a punctuation character results in an empty search.

In [6]:
def clean_char(token):
    punct_searches = []
    remove = [".", ",", "/", "?", "!", "$", "@", "&", "*", "(", ")", "#", "%", "^", "-", "_", "\"", "'", "[", "]"]
    for j in token:
        temp = j
        for k in remove:
            temp = temp.replace(k, "")
        if temp != '':
            punct_searches.append(temp)

    return punct_searches

The next cleaning function will be numbers. It is very similar to the char cleaning function, except it removes numbers instead. This function also omits empty searches.

In [7]:
def clean_num(token):
    num_searches = []
    remove = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]
    for j in token:
        temp = j
        for k in remove:
            temp = temp.replace(k, "")
        if temp != '':
            num_searches.append(temp)

    return num_searches

Now the lambda calls will be made for these two functions:

In [8]:
%%time
cleaned_chars = column.map(lambda search : clean_char(search))

Wall time: 3.18 s


In [9]:
%%time
cleaned_searches = cleaned_chars.map(lambda search : clean_num(search))

Wall time: 1.87 s


The new cleaned column will replace the old "dirty" column of searches.

In [10]:
df["Searches"] = cleaned_searches
df.head(15)

,Searches
0,[]
1,[cmed]
2,[kend]
3,[cmed]
4,[dynch]
5,[dynd]
6,"[dees, kc]"
7,"[link, pc]"
8,[]
9,"[kend, sa]"


From the display of the first 15 cleaned entries, one can see that there are numerous empty entires after the column was cleaned. These must be removed before any spell check is done so the spelling dictionary is not full of empty values. 

The following code removes any empty rows (since any empty list entries were removed, it can be assumed that any empty rows will just be an empty list).

In [11]:
%%time
df = df[df['Searches'].map(lambda d: len(d)) > 0]

Wall time: 287 ms


In [12]:
df.head(15)

,Searches
1,[cmed]
2,[kend]
3,[cmed]
4,[dynch]
5,[dynd]
6,"[dees, kc]"
7,"[link, pc]"
9,"[kend, sa]"
10,[kend]
11,[bacon]


Now that the dataframe is completly cleaned, a spellcheck dictionary can be made.

This process will be very similar to the one in the previous lab. First a spelling dictionary must be made from the dataframe. The following function and lines of code create a spell check dictionary where the possibly misspelled word is the key and the correctly spelled word is the value. 

In [13]:
def make_spelling_dict(clean_list):
    clean_dic = {"misspelled" : "correct"}
    spell = SpellChecker(distance = 1)
    for i in clean_list:
        for j in i:
            correct = spell.correction(j)
            clean_dic[j] = correct
    return clean_dic

diction = make_spelling_dict(df['Searches'])

NameError: name 'SpellChecker' is not defined

The next function looks at one token and replaces a miss spelled word with the correct spelling, if the word is not in the dictionary (spelled correctly), the original token is just returned.

In [14]:
def compare_to_dict(token, dicton):
    l = []
    for i in token:
        l.append(diction[str(i)])
    return l

Finally, the misspelled dataframe will be spellchecked by calling a lambda with the compare_to_dict function above. The return values will be put into a new dataframe and replace the old dataframe

In [15]:
%%time
df = df[df['Searches'].map(lambda d : compare_to_dict(d, diction))]

NameError: name 'diction' is not defined

In its current state, 'df' contains a cleaned, spell checked data frame with no empty rows. This data frame is now prepared for a term frequency dictionary. 
This program will explore two methods of creating a term frequency dictionary. The first will be identical to the approach used in lab 2. The second method will use value_counts method that is built into pandas. Both these methods will be benchmarked.

The following code will generate a python list from our data frame and print out the first few indises. 

In [16]:
data_list = df["Searches"].values.tolist()
print(data_list[0:15])

[['cmed'], ['kend'], ['cmed'], ['dynch'], ['dynd'], ['dees', 'kc'], ['link', 'pc'], ['kend', 'sa'], ['kend'], ['bacon'], ['pineapple'], ['enfit'], ['cheese', 'cheddar'], ['buttermilk'], ['milk']]


The function below will parse through a list with a nested for loop and create a term frequency dictionary. The first loop goes though each entry in the list, the second loop parses through each token (word) in a search. If the word is in the dictionary already, that words value is incrimented by one. If it is not in the dictioanary, the word is added to the end of the dictionary with a value of 1. A string builder is used to make a csv to display the dictionary.

In [17]:
def count_frequency(l):
    freq = {}
    for item in l:
        for k in item:
            if (k in freq):
                freq[k] += 1
            else:
                freq[k] = 1

    str_builder = ""
    for key, value in freq.items():
        t = str(key) + " , " + str(value)
        print(t)
        str_builder += t + "\n"

    return str_builder

Now the function will be called on our list and the dictionary will be printed out.

In [18]:
%%time
method_A_freq = count_frequency(data_list)
print(method_A_freq)

cmed , 4
kend , 189
dynch , 42
dynd , 2213
dees , 11
kc , 15
link , 922
pc , 601
sa , 133
bacon , 6198
pineapple , 3119
enfit , 189
cheese , 13957
cheddar , 1760
buttermilk , 457
milk , 4792
chicken , 19223
breast , 2900
drit , 22
romain , 135
banana , 3813
sl , 52
uroch , 2
huds , 11
name , 122
tags , 38
cut , 329
fruit , 5620
biscuit , 3622
way , 446
geri , 86
hcs , 1290
mash , 540
liquid , 2454
egg , 5930
wipes , 2124
dyndh , 227
creamer , 3418
lettuce , 2742
apple , 7843
sauce , 11288
cottage , 3579
lactose , 266
juice , 11452
org , 25
bottled , 88
water , 3949
pearls , 303
dinner , 1701
rolls , 2317
sausage , 7429
ct , 80
oil , 3769
white , 2803
bread , 7780
green , 9804
beans , 9137
kits , 144
avocado , 382
honeydew , 610
corn , 8171
flakes , 1112
coffee , 2514
panko , 319
magic , 594
cup , 3439
msc , 4940
wpw , 40
pang , 144
salad , 5952
mix , 7016
biscuits , 1180
patty , 3971
wheat , 2566
fitplusxlg , 85
base , 4812
mayo , 2868
ma , 8
russet , 104
ham , 5413
pine , 205
napkin ,

For the count_values method, all that needs to be done is to call the method on the correct series in the dataframe

In [ ]:
%%time
method_B_freq = df['Searches'].value_counts()
print(method_B_freq)

### Dicussion on benchmarking

Taking a step back from the two term freuqency dictionaries, this program now has a python list and pandas dataframe with the same information. The following code prints out the memory usage for each structure.

In [28]:
def get_mem_list(l):
    total = 0
    for i in l:
        total = total + sys.getsizeof(i)
    return total


list_mem = get_mem_list(data_list)
df_mem = df.memory_usage(deep=True)

print("Memory size of list is " + str(list_mem / 1000) + " kb")
print("Memory size of dataframe is " + str(df_mem / 1000) + " kb")

Memory size of list is 78596.848 kb
Memory size of dataframe is Index        7144.848
Searches    71452.000
dtype: float64 kb


Ignoring all the mumbo jumbo that results from the data frame memory call, one can observe that the python list uses roughly 10 times the amount of memory as the dataframe. Although I knew pythons built in data structures had poor memory footprints, I did not expect it to be to this magnitude. From this observation, one can understand why pandas are used when dealing with large datasets.

# conclusion